In [ ]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

BATCH_SIZE = 100
WEIGHT_DECAY = 0.005
LEARNING_RATE = 0.0001
EPOCH = 100
PATH = "C:\\Users\\arimoto\\ObjectOperate"
#TRAINPATH = "C:\\Users\\arimoto\\cnntest\\train"
#TESTPATH = "C:\\Users\\arimoto\\cnntest\\test"
#VALIDPATH = "C:\\Users\\arimoto\\cnntest\\valid"

image_size=36
#object_name = "object4"

datadirs = glob.glob(PATH + "\\dataset\\*")
imgfiles = glob.glob(PATH + "\\" + object_name + "\\"+ object_name + ".png")
csvdatafiles = glob.glob(PATH + "\\" + object_name + "\\data\\*.csv")
csvlabelfiles = glob.glob(PATH + "\\" + object_name + "\\label\\*.csv")

img_arrays = []
img_names = []
datas = []
labels = []

for imgfile in imgfiles:
    # 画像読み込み
    image = Image.open(imgfile)
    # グレイスケール変換
    image = image.convert('L')
    # リサイズ
    #image = image.resize((image_size, image_size))
    # 画像から配列に変換
    img_data = np.asarray(image)
    img_names.append(os.path.basename(imgfile))
    #file_split = [i for i in file.split('_')]
    img_arrays.append(img_data)
    
for datafile in csvdatafiles:
    with open(datafile) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                points = []
                for point in row:
                    points.append(int(point))
                datas.append(np.asarray(points))
                num += 1
for labelfile in csvlabelfiles:
    with open(labelfile) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                #print(row)
                points = []
                for point in row:
                    points.append(int(point))
                labels.append(np.asarray(points))
                num += 1
#labels.append((int(file_split[2]), int(file_split[3])))
    
print("fin loading")
datas = np.array(datas)
labels = np.array(labels)
img_arrays = np.array(img_arrays)
print("data" + str(datas))
print("label" + str(labels))
print("image_names" + str(img_names))
#print("image_arrays" + str(img_arrays))
#for i in range(img_arrays.shape[1]):
#    print(img_arrays.shape)
#    print(img_arrays[0][i])


# In[3]:


# trainデータとtestデータに分割
data_train, data_test, label_train, label_test = train_test_split(
    datas,
    labels,
    random_state = 0,
    test_size = 0.2
)
print(data_train.shape, label_train.shape, data_test.shape, label_test.shape)


# In[4]:


# データ型の変換＆正規化
data_train = data_train.astype('float32')# / 255
data_test = data_test.astype('float32')# / 255
img_array = img_arrays[0].astype('float32')/255


# In[5]:


trans = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

print(data_train[0])
print(label_train[0])
print(data_test[0])
print(label_test[0])
print(img_array)


# In[6]:


class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, datas, labels, img_array, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.label = labels
        self.img_array = img_array

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        i_data = self.data[idx]
        i_label = self.label[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_label = np.array(i_label.astype(np.float32))
        out_data = np.array(i_data.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(self.img_array)
            #out_label = self.transform2(out_label)

        return out_data, out_label, out_img


# In[7]:


trainset = Mydatasets(datas = data_train, labels = label_train, img_array = img_array, transform = trans)
testset = Mydatasets(datas = data_test, labels = label_test, img_array = img_array, transform = trans)


# In[8]:


trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)


# In[9]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, stride=2)

        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)

        self.fc1 = nn.Linear(32 * 7 * 7 + 11, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x, y):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size()[0], -1)
        #print("x_shape: ", x.size())
        #print("y_shape: ", y.size())
        x = torch.cat([x, y], axis = -1)
        #print("torch.cat(x, y): ", x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# In[10]:


device = torch.device("cuda:0")
net = Net()
net = net.to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
#train_acc_value=[]       #trainingのaccuracyを保持するlist
test_loss_value=[]       #testのlossを保持するlist
#test_acc_value=[]        #testのaccuracyを保持するlist 

max_train_loss_value = 0.
max_test_loss_value = 0.

for epoch in range(EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    for (inputs, labels, input_img) in trainloader:
        inputs, labels, input_img = inputs.to(device), labels.to(device), input_img.to(device)
        optimizer.zero_grad()
        #print("inputs:", inputs)
        outputs = net(input_img, inputs)
        #if epoch%10 == 0:
            #print("outputs:", outputs)
            #print("labels:", labels)
        
#        out_sums = []
#        lab_sums = []
#        for out in outputs:
#            out_sums.append(out[0].item()+out[1].item())
#        for lab in labels:
#            lab_sums.append(lab[0].item()+lab[1].item())
#        print(out_sums)
#        print(lab_sums)
#        crit_outputs = torch.tensor(np.array(out_sums))
#        crit_labels = torch.tensor(np.array(lab_sums))
        
        loss = criterion(outputs, labels)#crit_outputs, crit_labels)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    #sum_correct = 0         #正解率の合計
    sum_total = 0           #dataの数の合計

    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels, input_img) in trainloader:
        inputs, labels, input_img = inputs.to(device), labels.to(device), input_img.to(device)
        optimizer.zero_grad()
        outputs = net(input_img, inputs)

        loss = criterion(outputs, labels)
        sum_loss += loss.item()                            #lossを足していく
        #_, predicted = outputs.max(1)                      #出力の最大値の添字(予想位置)を取得
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
        #sum_correct += (predicted == labels).sum().item()  #予想位置と実際の正解を比べ,正解している数だけ足す
    print("train mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(trainloader.dataset)))  #loss出力
    train_loss_value.append(sum_loss*BATCH_SIZE/len(trainloader.dataset))  #traindataのlossをグラフ描画のためにlistに保持
    #train_acc_value.append(float(sum_correct/sum_total))   #traindataのaccuracyをグラフ描画のためにlistに保持
    if sum_loss*BATCH_SIZE/len(trainloader.dataset) > max_train_loss_value:
        max_train_loss_value = sum_loss*BATCH_SIZE/len(trainloader.dataset)

    sum_loss = 0.0
    sum_correct = 0
    sum_total = 0

    #test dataを使ってテストをする
    for (inputs, labels, input_img) in testloader:
        inputs, labels, input_img = inputs.to(device), labels.to(device), input_img.to(device)
        optimizer.zero_grad()
        outputs = net(input_img, inputs)

        loss = criterion(outputs, labels)
        sum_loss += loss.item()
        #_, predicted = outputs.max(1)
        sum_total += labels.size(0)
        #sum_correct += (predicted == labels).sum().item()
    print("test mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(testloader.dataset)))
    test_loss_value.append(sum_loss*BATCH_SIZE/len(testloader.dataset))
    #test_acc_value.append(float(sum_correct/sum_total))
    if sum_loss*BATCH_SIZE/len(testloader.dataset) > max_test_loss_value:
        max_test_loss_value = sum_loss*BATCH_SIZE/len(testloader.dataset)

plt.figure(figsize=(6,6))      #グラフ描画用
ylim = max(max_train_loss_value, max_test_loss_value)

#以下グラフ描画
plt.plot(range(EPOCH), train_loss_value)
plt.plot(range(EPOCH), test_loss_value, c='#00ff00')
plt.xlim(0, EPOCH)
plt.ylim(0, ylim)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig(PATH + "\\" + object_name + "\\loss_image.png")
plt.clf()


# In[10]:


a = torch.ones((16, 5))
b = torch.zeros((16, 3))


# In[30]:


torch.cat([a, b], axis=-1).shape


# In[18]:


#input_img_test = trans(img_array)
#print(input_img_test)


# In[11]:


import random

label_points = []
label_view = []

for l in range(10):
    label_points.append(0)
label_view.append(label_points)
label_view = np.array(label_view)

def view_sequence(points):
    data_view = []
    data_view.append(points)
    data_view = np.array(data_view)
    #print(data_view.shape, label_view.shape)
    data_view = data_view.astype('float32')# / 255
    #init_l_points = np.array(init_l_points.astype(np.float32))
    #print(init_l_points)
    viewset = Mydatasets(datas = data_view, labels = label_view, img_array = img_array, transform = trans)
    viewloader = torch.utils.data.DataLoader(viewset, batch_size = 1, shuffle = False, num_workers = 0)
    
    for (inputs, labels, input_img) in viewloader:
        inputs, labels, input_img = inputs.to(device), labels.to(device), input_img.to(device)
        optimizer.zero_grad()
        view_out = net(input_img, inputs)
        #print(view_out)
    view_out_np = view_out.to('cpu').detach().numpy().copy()
    #print(view_out_np[0])
    return view_out_np[0]


# In[14]:


init_l_points = []

#5つのランダムな点を作成(x, y)
fin1x = random.randint(-80, -11)
fin2x = random.randint(-80, -11)
fin3x = random.randint(-80, -11)
fin4x = random.randint(-80, -11)
fin5x = random.randint(-80, -11)
fin1y = random.randint(-80, 80)
fin2y = random.randint(-80, 80)
fin3y = random.randint(-80, 80)
fin4y = random.randint(-80, 80)
fin5y = random.randint(-80, 80)
init_l_points.append(fin1x)
init_l_points.append(fin1y)
init_l_points.append(fin2x)
init_l_points.append(fin2y)
init_l_points.append(fin3x)
init_l_points.append(fin3y)
init_l_points.append(fin4x)
init_l_points.append(fin4y)
init_l_points.append(fin5x)
init_l_points.append(fin5y)
init_l_points.append(0)
l_points = np.asarray(init_l_points)
print(l_points)

with open(PATH + "\\" + object_name + "\\view_l_0.csv" , 'w') as f:
    writer = csv.writer(f)
    writer.writerow(l_points.tolist())
    
for v in range(10):
    l_points = view_sequence_flip(l_points)
    #print(l_points)
    l_points = l_points.tolist()
    l_points.append(v+1)
    print(l_points)
    with open(PATH + "\\" + object_name + "\\view_l_" + str(v+1) + ".csv" , 'w') as f:
        writer = csv.writer(f)
        writer.writerow(l_points)
    l_points = np.array(l_points)


# In[15]:


init_r_points = []

#5つのランダムな点を作成(x, y)
fin1x = random.randint(20, 80)
fin2x = random.randint(20, 80)
fin3x = random.randint(20, 80)
fin4x = random.randint(20, 80)
fin5x = random.randint(20, 80)
fin1y = random.randint(-80, 80)
fin2y = random.randint(-80, 80)
fin3y = random.randint(-80, 80)
fin4y = random.randint(-80, 80)
fin5y = random.randint(-80, 80)
init_r_points.append(fin1x)
init_r_points.append(fin1y)
init_r_points.append(fin2x)
init_r_points.append(fin2y)
init_r_points.append(fin3x)
init_r_points.append(fin3y)
init_r_points.append(fin4x)
init_r_points.append(fin4y)
init_r_points.append(fin5x)
init_r_points.append(fin5y)
init_r_points.append(0)
r_points = np.asarray(init_r_points)
print(r_points)

with open(PATH + "\\" + object_name + "\\view_r_0.csv" , 'w') as f:
    writer = csv.writer(f)
    writer.writerow(r_points.tolist())

for v in range(10):
    r_points = view_sequence_flip(r_points)
    #print(l_points)
    r_points = r_points.tolist()
    r_points.append(v+1)
    print(r_points)
    with open(PATH + "\\" + object_name + "\\view_r_" + str(v+1) + ".csv" , 'w') as f:
        writer = csv.writer(f)
        writer.writerow(r_points)
    r_points = np.array(r_points)


# In[13]:


from PIL import ImageOps

img_arrays_flip = []
imgfile_flip = PATH + "\\" + object_name + "\\"+ object_name + ".png"

# 画像読み込み
image_flip = Image.open(imgfile_flip)
#左右反転処理
image_flip = ImageOps.mirror(image_flip)
# グレイスケール変換
image_flip = image_flip.convert('L')
# リサイズ
#image = image.resize((image_size, image_size))
# 画像から配列に変換
img_data_flip = np.asarray(image_flip)
#img_names.append(os.path.basename(imgfile))
#file_split = [i for i in file.split('_')]
img_arrays_flip.append(img_data_flip)
img_arrays_flip = np.array(img_arrays_flip)
img_array_flip = img_arrays_flip[0].astype('float32')/255

def view_sequence_flip(points):
    data_view = []
    data_view.append(points)
    data_view = np.array(data_view)
    #print(data_view.shape, label_view.shape)
    data_view = data_view.astype('float32')# / 255
    #init_l_points = np.array(init_l_points.astype(np.float32))
    #print(init_l_points)
    viewset = Mydatasets(datas = data_view, labels = label_view, img_array = img_array_flip, transform = trans)
    viewloader = torch.utils.data.DataLoader(viewset, batch_size = 1, shuffle = False, num_workers = 0)
    
    for (inputs, labels, input_img) in viewloader:
        inputs, labels, input_img = inputs.to(device), labels.to(device), input_img.to(device)
        optimizer.zero_grad()
        view_out = net(input_img, inputs)
        #print(view_out)
    view_out_np = view_out.to('cpu').detach().numpy().copy()
    #print(view_out_np[0])
    return view_out_np[0]